In [1]:
from __future__ import print_function

import os, sys
module_path = os.path.abspath(os.path.join('../../..'))
sys.path.append(module_path)

import numpy as np
import math
import copy
import pandas as pd
from keras.utils import np_utils
from keras.datasets import mnist
import time
import pickle

from pycrcnn.he.he import TFHEnuFHE
from pycrcnn.he.tfhe_value import TFHEValue
from pycrcnn.he.alu import *

2023-01-24 23:32:24.589266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 23:32:24.893013: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 23:32:26.236568: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-24 23:32:26.236642: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

## HE Init

In [2]:
HE_client = TFHEnuFHE(22)

with open("res/secret_key", "rb") as f:
    HE_client.secret_key = HE_client.ctx.load_secret_key(f)
    
with open("res/cloud_key", "rb") as f:
    HE_client.cloud_key = HE_client.ctx.load_cloud_key(f)

cloud_key = HE_client.cloud_key
HE_client.generate_vm(cloud_key)

In [3]:
num1 = HE_client.encrypt(1)
num2 = HE_client.encode(6)
sum = num1+num2
mul = num1*num2

## Experiments

In [4]:
## UPLOAD DFA WEIGHTS
DFA_weights1 = np.load("res/DFA_weights_L1.npy")
DFA_weights2 = np.load("res/DFA_weights_L2.npy")

### Aggregation

In [5]:
# Load decrypted trained weights model 1
with open("res/trained_weights_M1.pkl", "rb") as f:
    weights2_M1 = pickle.load(f)
    bias2_M1 = pickle.load(f)
    weights3_M1 = pickle.load(f)
    bias3_M1 = pickle.load(f)
    weights4_M1 = pickle.load(f)
    bias4_M1 = pickle.load(f)

In [6]:
# Load decrypted trained weights model 2
with open("res/trained_weights_M2.pkl", "rb") as f:
    weights2_M2 = pickle.load(f)
    bias2_M2 = pickle.load(f)
    weights3_M2 = pickle.load(f)
    bias3_M2 = pickle.load(f)
    weights4_M2 = pickle.load(f)
    bias4_M2 = pickle.load(f)

In [7]:
# Group encrypted weights
W2 = [HE_client.encrypt_matrix(weights2_M1), HE_client.encrypt_matrix(weights2_M2)]
B2 = [HE_client.encrypt_matrix(bias2_M1), HE_client.encrypt_matrix(bias2_M2)]

W3 = [HE_client.encrypt_matrix(weights3_M1), HE_client.encrypt_matrix(weights3_M2)]
B3 = [HE_client.encrypt_matrix(bias3_M1), HE_client.encrypt_matrix(bias3_M2)]

W4 = [HE_client.encrypt_matrix(weights4_M1), HE_client.encrypt_matrix(weights4_M2)]
B4 = [HE_client.encrypt_matrix(bias4_M1), HE_client.encrypt_matrix(bias4_M2)]

In [8]:
%%time
# Aggragation of encrypted weights
average_weights2 = encrypted_mean_matrix(W2)
average_bias2 = encrypted_mean_matrix(B2)
average_weights3 = encrypted_mean_matrix(W3)
average_bias3 = encrypted_mean_matrix(B3)
average_weights4 = encrypted_mean_matrix(W4)
average_bias4 = encrypted_mean_matrix(B4)

CPU times: user 11min 41s, sys: 223 ms, total: 11min 41s
Wall time: 27min 51s


### Serialization

In [9]:
# Decrypt aggregated encrypted weights
decrypted_weights2 = HE_client.decrypt_matrix(average_weights2)
decrypted_bias2 = HE_client.decrypt_matrix(average_bias2)
decrypted_weights3 = HE_client.decrypt_matrix(average_weights3)
decrypted_bias3 = HE_client.decrypt_matrix(average_bias3)
decrypted_weights4 = HE_client.decrypt_matrix(average_weights4)
decrypted_bias4 = HE_client.decrypt_matrix(average_bias4)

In [10]:
# Save decrypted trained weights
with open("res/aggregated_weights.pkl", "wb") as f:
    pickle.dump(decrypted_weights2, f)
    pickle.dump(decrypted_bias2, f)
    pickle.dump(decrypted_weights3, f)
    pickle.dump(decrypted_bias3, f)
    pickle.dump(decrypted_weights4, f)
    pickle.dump(decrypted_bias4, f)